# Demo notebook: Independent Component Analysis to remove stripes from the calcium imaging recordings

In [ ]:
import os
import sys

sys.path.append('/Users/annateruel/phd_code/')
from ca2img.icas import VideoPCA
from ca2img.minian_dca1 import start_cluster

In [ ]:
start_cluster(n_workers=8, memory_limit="6GB")

In [ ]:
dir = '/Volumes/ANNA_HD/ANALYSIS/Ca2+Img/23-03-ST-anna/AD22-265/annateruel/srt_trial/AD22-265/minian/2023_04_13/16_50_31/My_V4_Miniscope'
avi_files = [os.path.join(dir, f) for f in os.listdir(dir) if f.endswith('.avi') and not f.startswith('p_')]
avi_files.sort()
avi_files

In [ ]:
video_pca = VideoPCA(avi_files)
video_pca.collect_frames()

In [ ]:
video_pca.pca_videos()

In [ ]:
video_pca.plot_cumulative_variance()

Typically the number of ICA can be based on the number of principal components that explain a significant amount of variance in my data. For instance, we should chose the number of components that explain at least 90% of the variance. 

# TRIAL


In [21]:
import numpy as np
import cv2
import os
import subprocess
import json

def get_video_properties(video_path):
    command = [
        'ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries',
        'stream=width,height,r_frame_rate', '-of', 'json', video_path
    ]
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        raise ValueError(f"Failed to get video properties: {result.stderr}")
    
    properties = json.loads(result.stdout)
    width = properties['streams'][0]['width']
    height = properties['streams'][0]['height']
    fps = eval(properties['streams'][0]['r_frame_rate'])  # Evaluate to get FPS as a float
    return width, height, fps

class ICAProcessor:
    def __init__(self, sources, components, frame_shape, fps):
        self.sources = sources  # Temporal components
        self.components = components  # Spatial components
        self.frame_shape = frame_shape
        self.fps = fps
    
    def reconstruct_from_discarded_components(self, discarded_components):
        """Reconstruct data excluding discarded components."""
        if self.sources is None or self.components is None:
            raise ValueError("ICA not performed. Please provide the sources and components.")
        
        # Identify non-discarded components
        all_components = np.arange(self.components.shape[0])
        selected_components = np.setdiff1d(all_components, discarded_components)
        
        selected_sources = self.sources[:, selected_components]
        selected_components = self.components[selected_components, :]
        
        # Multiply the temporal and spatial components and sum them
        reconstructed_data = np.dot(selected_sources, selected_components)
        
        return reconstructed_data

    def save_reconstructed_video(self, cleaned_data, output_file):
        """Save the reconstructed data as a video using OpenCV."""
        height, width = self.frame_shape
        
        print(f"Frame shape: {self.frame_shape}")  # Debug statement
        print(f"Cleaned data shape: {cleaned_data.shape}")  # Debug statement
        
        # Verify the cleaned_data shape matches the expected number of frames and frame size
        num_frames, data_size = cleaned_data.shape
        expected_data_size = height * width
        if data_size != expected_data_size:
            raise ValueError(f"Data size mismatch: expected {expected_data_size}, got {data_size}")
        
        # Define the codec and create VideoWriter object for AVI
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(output_file, fourcc, self.fps, (width, height), isColor=False)
        
        for i, frame in enumerate(cleaned_data):
            try:
                frame_data = frame.reshape(self.frame_shape)
                frame_data = cv2.normalize(frame_data, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
                frame_data = cv2.cvtColor(frame_data, cv2.COLOR_GRAY2BGR)  # Convert to 3 channels
                out.write(frame_data)
            except Exception as e:
                print(f"Error processing frame {i}: {e}")
        
        out.release()
        print(f"Saved reconstructed video to {output_file}")

    def convert_video_format(self, input_file, output_file):
        """Convert video to a different format using ffmpeg."""
        command = [
            'ffmpeg',
            '-i', input_file,
            '-c:v', 'libx264',
            '-preset', 'slow',
            '-crf', '22',
            output_file
        ]
        try:
            subprocess.run(command, check=True)
            print(f"Converted video saved to {output_file}")
        except subprocess.CalledProcessError as e:
            print(f"Error converting video: {e}")

# Example usage
if __name__ == "__main__":
    frame_shape = (484, 548)

    # Load the necessary data
    ica_sources = np.load('/Users/annateruel/output_plots/ica_sources.npy')
    ica_components = np.load('/Users/annateruel/output_plots/ica_components.npy')
    # Mean is not loaded as it won't be used

    # Initialize the ICAProcessor
    processor = ICAProcessor(sources=ica_sources, components=ica_components, frame_shape=frame_shape, fps=60)

    # List of discarded component indices (0-based)
    discarded_components = [0, 1, 3, 4, 6, 7, 10, 11, 13, 16, 18, 27, 28, 30, 32, 33, 34, 35, 36, 38, 39, 41, 45, 48, 46, 58, 61, 63, 68, 78, 86, 96, 98, 101, 104, 105, 107, 111, 113, 117, 118, 121, 122, 123, 124, 125, 130, 138, 140, 143, 144, 148, 154, 157, 160, 165, 173, 172, 175, 178, 182, 181, 175, 178, 181, 182, 183, 185, 192]

    # Reconstruct the data excluding discarded components
    cleaned_data = processor.reconstruct_from_discarded_components(discarded_components)

    # Save the reconstructed video
    output_dir =  '/Users/annateruel/Desktop/My_V4_Miniscope/'
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(output_dir, 'reconstructed_video.avi')
    processor.save_reconstructed_video(cleaned_data, output_file)

    # Convert the video format for compatibility if needed
    converted_output_file = os.path.join(output_dir, 'reconstructed_video_converted.mp4')
    processor.convert_video_format(output_file, converted_output_file)


Frame shape: (484, 548)
Cleaned data shape: (1000, 265232)
Saved reconstructed video to /Users/annateruel/Desktop/My_V4_Miniscope/reconstructed_video.avi


ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e

Error converting video: Command '['ffmpeg', '-i', '/Users/annateruel/Desktop/My_V4_Miniscope/reconstructed_video.avi', '-c:v', 'libx264', '-preset', 'slow', '-crf', '22', '/Users/annateruel/Desktop/My_V4_Miniscope/reconstructed_video_converted.mp4']' returned non-zero exit status 183.


[out#0/mp4 @ 0x123638420] Nothing was written into output file, because at least one of its streams received no packets.
frame=    0 fps=0.0 q=0.0 Lsize=       0KiB time=N/A bitrate=N/A speed=N/A    
Conversion failed!
